<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/JuliaRQIQuadracticOnSphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> 
Colab Notebook for Generalized Rayleigh Quotient Iteration and Rayleigh Chebyshev Iteration to find solutions of the equation
$\newcommand{\R}{\mathbb{R}}$
$$ Ax -\lambda x -b = 0\\
x^Tx = 1
$$
for a matrix $A\in \R^{n\times n}, b\in\R^n, \lambda\in \R$, not necessarily symmetric.
* When $A$ is symmetric, it gives us the set of critical points of the quadratic function $\frac{1}{2}x^TAx - bx$.

#  Summary: Confirming RQI has quadratic, Rayleigh-Chebyshev has cubic convergence order.
* An example: for $n=10, A\in \R^{n\times n}, b\in \R^n$, at the warm up point with residual error 4.615e-03, the sequence of residual error for Rayleight and Rayleigh Chebyshev are:


|Iteration | Rayleigh | Rayleigh Chebyshev|
|---|---|---|
|1|6.986e-06|6.986e-06|
|2|2.368e-11|6.511e-15|
|3|3.500e-22|5.208e-42|
|4|9.982e-44| |

* Use the library Arblib for higher numerical precision to demonstrate cubic convergence.

# You can view the results as-is. If you want to run the code follow the instructions below
## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). **This takes a couple of minutes.**
4. **Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.**

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-02-13 19:08:27 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


# Checking the Installation
**REMEMBER TO LOAD THE PAGE BY RUNNING F5 IF the following command does not work**

The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver2)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Use Arblib to test the package to show higher precision

Which demostrates Chebyshev has cubic convergence and Newton has quadratic convergence

In [2]:
using Pkg
Pkg.add("Arblib")
using Arblib
using LinearAlgebra
using Printf
import Random



    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed FLINT_jll ─────────── v200.900.4+0
   Installed IrrationalConstants ─ v0.1.1
   Installed Arb_jll ───────────── v200.2300.0+0
   Installed OpenBLAS32_jll ────── v0.3.17+0
   Installed OpenSpecFun_jll ───── v0.5.5+0
   Installed SpecialFunctions ──── v2.1.7
   Installed Compat ────────────── v4.6.0
   Installed LogExpFunctions ───── v0.3.21
   Installed Arblib ────────────── v0.8.1
   Installed ChainRulesCore ────── v1.15.7
   Installed ChangesOfVariables ── v0.1.5
   Installed InverseFunctions ──── v0.1.8
   Installed DocStringExtensions ─ v0.9.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [fb37089c] + Arblib v0.8.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [fb37089c] + Arblib v0.8.1
  [d360d2e6] + ChainRulesCore v1.15.7
  [9e997f8a] + ChangesOfVariables v0.1.5
  [34da2185] + Compat v4.6.0
  [ffbed154] + DocStringExtensions v0.9.3
  [3587e190] + Inverse

# a few functions to generate random symmetric tensors

In [3]:
using Arblib
using LinearAlgebra
using Printf
import Random


function run_one_group(x0, A, b)
  function Newton(x)
      lbd = sum(x.*(A*x-b))
      Lx = A - lbd*I
      luLx = lu(Lx)
      ldiv!(rbx, luLx, hcat(b, x))
          
      ft = (1 - sum(x.*rbx[:, 1]))/sum(x.*rbx[:, 2])
      return (rbx[:, 1] + rbx[:, 2]*ft)/norm(rbx[:, 1] + rbx[:, 2]*ft), lbd
  end

  function Chebyshev(x)
      lbd = sum(x.*(A*x-b))        
      Lx[:] = A - lbd*I
      chev1 = ArbMatrix(zeros(n, 1), prec=prec)
      luLx = lu(Lx)
      ldiv!(rbx, luLx, hcat(b, x))        
      ft = (1 - sum(x.*rbx[:, 1]))/sum(x.*rbx[:, 2])    
      eta = - x + rbx[:, 1] + rbx[:, 2]*ft
      Rp = sum(eta.*((A+A')*x .- b))   

      ldiv!(chev1, Lx, reshape(eta, n, 1))
      chev1 .*= -2*Rp
      chev1 .-= sum(eta.*eta)*x
      
      chev =  chev1 .- sum(x.*chev1)/sum(x.*rbx[:, 2])*rbx[:, 2]
      etaC = eta .- 0.5*chev
      return (x+etaC)/norm(x+etaC), lbd
  end    

  for i in 1:10
      x0, lbd = Newton(x0)
      diffs = A*x0 .- lbd*x0 .- b
      dch = sum(diffs.*diffs)^0.5
      @printf("WARM UP %.3e\n", sum(diffs.*diffs)^0.5)
      if isnan(dch) | (dch < 5e-2)
          break
      end
  end

  x = copy(x0)
  for i in 1:20
      x, lbd = Newton(x)
      diffs = A*x .- lbd*x .- b
      dch = sum(diffs.*diffs)^0.5
      @printf("RAYLEIGH %.3e\n", dch)
      if isnan(dch) | (dch < 1e-30)
          break
      end
  end

  x = copy(x0)
  for i in 1:20
      x, lbd = Chebyshev(x)
      diffs = A*x .- lbd*x .- b
      dch = sum(diffs.*diffs)^0.5
      @printf("RAYLEIGH-CHEBYSHEV %.3e\n", dch)
      if isnan(dch) | (dch < 1e-30)
          break
      end
  end
end


run_one_group (generic function with 1 method)

In [11]:
Random.seed!(0)
prec = 512
n = 10

A = ArbMatrix(Random.rand(n, n); prec=prec)


b = ArbMatrix(Random.rand(n), prec=prec)

rbx = ArbMatrix(zeros(n, 2), prec=prec)
Lx = copy(A)
x0 = ArbMatrix(Random.rand(n), prec=prec)
run_one_group(x0, A, b)


WARM UP 6.935e+00
WARM UP 1.177e-01
WARM UP 4.615e-03
RAYLEIGH 6.986e-06
RAYLEIGH 2.368e-11
RAYLEIGH 3.500e-22
RAYLEIGH 9.982e-44
RAYLEIGH-CHEBYSHEV 6.986e-06
RAYLEIGH-CHEBYSHEV 6.511e-15
RAYLEIGH-CHEBYSHEV 5.208e-42
